In [1]:
import cv2
import mediapipe as mp
import numpy as np
from math import hypot
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
camSrc = cv2.VideoCapture(0)
handLoc = mp.solutions.hands
HLoc = handLoc.Hands()
mpDraw = mp.solutions.drawing_utils


devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

volMin,volMax = volume.GetVolumeRange()[:2]
while True:
    index,img = camSrc.read()
    imgRGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    

    results = HLoc.process(imgRGB)
    markerlist = []
    if results.multi_hand_landmarks:
        for handlandmark in results.multi_hand_landmarks:
            for id,landmark in enumerate(handlandmark.landmark):

                height,width,_ = img.shape
                LMx,LMy = int(landmark.x*width),int(landmark.y*height)
                markerlist.append([id,LMx,LMy])
            mpDraw.draw_landmarks(img,handlandmark,handLoc.HAND_CONNECTIONS)
    if markerlist != []:

        x1,y1 = markerlist[4][1],markerlist[4][2]
        x2,y2 = markerlist[20][1],markerlist[20][2]
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3) 
        
        length = hypot(x2-x1,y2-y1) 
        vol = np.interp(length,[15,220],[volMin,volMax])
#         print(vol,length)
        volume.SetMasterVolumeLevel(vol, None)

    cv2.imshow('Video Input',img) 
    if cv2.waitKey(1) & 0xff==ord(' '):
        break
camSrc.release()     
cv2.destroyAllWindows()